In [11]:
from numpy.core.numeric import roll
from solid.objects import circle, cube, cylinder, linear_extrude, minkowski, offset, rotate, rotate_extrude, sphere, square, translate, import_scad, hull
from solid.utils import right, up, forward
from viewscad import Renderer


r = Renderer()
r.width = 600
r.height = 600

render = r.render
PI = 3.14


def spin(obj, amount=3):
    return sum((rotate((0, 0, a))(obj) for a in range(0, 360, 360//amount)))


def get_roll_offset(d, w, gap, roll):
    return (d-w-gap-roll)/2


def pump_shell(d, w, h, tube, pin=6):
    inlet_offset = (d - tube)/2 - w
    inlet = rotate((0, 90, 0))(cylinder(d=tube, h=d, segments=100))

    return cylinder(d=d, h=h, segments=100)\
        - cylinder(d=d-w*2, h=h, segments=100)\
        + cylinder(d=d, h=w, segments=100)\
        + cylinder(d=pin, h=h+w, segments=100)\
        - translate((0, inlet_offset, h/2))(inlet)\
        - translate((0, -inlet_offset, h/2))(inlet)\



def pump_rolls(d, w, h, amount=3, roll=20, gap=0.2):
    offset = get_roll_offset(d, w, gap, roll)
    roll = cylinder(d=roll, h=h, segments=100)
    rolls = spin(right(offset)(roll), amount)

    return rolls


def pump_spinner(d, w, h, amount=3, roll=20, gap=0.2, hole=6):
    offset = get_roll_offset(d, w, gap, roll)
    size = d-w-gap
    cutout = cylinder(d=roll+gap, h=h, segments=100)
    cutouts = spin(right(offset)(cutout), amount)
    trim = rotate((0,0,180))(cylinder(d=size, h=h, segments=amount))

    spinner = up(w)(hull()(cutouts) - cutouts)
    spinner -= up(w)(cylinder(d=hole+gap, h=h, segments=100))
    spinner *= up(w)(trim)

    spinner += cylinder(d=size, h=w, segments=100)\

    return spinner


def peristaltic_pump(d=60, tube=12.5, tube_wall=2.5):
    height = PI*tube/2  # max size of squished tube
    padding = tube_wall*2
    shell = pump_shell(d=d, w=tube_wall, h=height+padding, tube=tube)
    spinner = pump_spinner(d=d-tube_wall*2, w=tube_wall, h=height)
    rolls = pump_rolls(d=d-tube_wall*2, w=tube_wall, h=height)

    return (shell, spinner, rolls)


(shell, spinner, rolls) = peristaltic_pump()

render(spinner - rolls)

render(shell, outfile='./outfiles/peristaltic_pump_shell.stl')
render(spinner, outfile='./outfiles/peristaltic_pump_spinner.stl')
render(rolls, outfile='./outfiles/peristaltic_pump_rolls.stl')


In [3]:
from numpy.core.numeric import roll
from solid.objects import circle, cube, cylinder, linear_extrude, minkowski, offset, rotate, rotate_extrude, sphere, square, translate, import_scad, hull
from solid.utils import right, up, forward
from viewscad import Renderer


r = Renderer()
r.width = 600
r.height = 600

render = r.render


def gear_belt(d=100, h=10, tooth_thickness=2, tooth_depth=2, teeth_spacing=4):
    ring = cylinder(segments=100, d=d, h=h) -cylinder(segments=100, d=d-1, h=h)
    tooth_base = cube((1,tooth_thickness,h))
    tooth_peak = translate((tooth_depth,0,tooth_depth))(cube((1,tooth_thickness,h-tooth_depth*2)))
    tooth = translate((-d/2,-tooth_thickness/2,0))(hull()(tooth_base, tooth_peak))

    tooth_footprint = tooth_thickness+teeth_spacing
    teeth = sum((rotate((0,0,z))(tooth) for z in range(0,360, tooth_footprint)))
    
    return ring + teeth

render(gear_belt(), outfile='./outfiles/gearbelt_test.stl')

/home/gitpod/.pyenv/versions/3.8.12/lib/python3.8/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
